In [ ]:
import os
print(os.listdir('D:/17flowers'))

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import matplotlib.pyplot as plt

#model selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#preprocess.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import VGG16


# specifically for cnn
from tensorflow.keras.layers import Dropout, Flatten,Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os


X=[]
Z=[]
IMG_SIZE=150
FLOWER_bluebell_DIR='D:/17flowers/bluebell'
FLOWER_buttercup_DIR='D:/17flowers/buttercup'
FLOWER_coltsfoot_DIR='D:/17flowers/coltsfoot'
FLOWER_cowslip_DIR='D:/17flowers/cowslip'
FLOWER_crocus_DIR='D:/17flowers/crocus'
FLOWER_Daffodil_DIR='D:/17flowers/Daffodil'
FLOWER_Daisy_DIR='D:/17flowers/Daisy'
FLOWER_dandelion_DIR='D:/17flowers/dandelion'
FLOWER_fritillary_DIR='D:/17flowers/fritillary'
FLOWER_iris_DIR='D:/17flowers/iris'
FLOWER_lilyvalley_DIR='D:/17flowers/lilyvalley'
FLOWER_pansy_DIR='D:/17flowers/pansy'
FLOWER_Snowdrop_DIR='D:/17flowers/Snowdrop'
FLOWER_sunflower_DIR='D:/17flowers/sunflower'
FLOWER_tigerlily_DIR='D:/17flowers/tigerlily'
FLOWER_tulip_DIR='D:/17flowers/tulip'
FLOWER_windflower_DIR='D:/17flowers/windflower'

def assign_label(img,flower_type):
    return flower_type

def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        try:
            label=assign_label(img,flower_type)
            path = os.path.join(DIR,img)
            img = cv2.imread(path,cv2.IMREAD_COLOR)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
            X.append(np.array(img))
            Z.append(str(label))

        except Exception as e:
            print(str(e))

make_train_data('bluebell',FLOWER_bluebell_DIR)
print(len(X))

make_train_data('buttercup',FLOWER_buttercup_DIR)
print(len(X))

make_train_data('coltsfoot',FLOWER_coltsfoot_DIR)
print(len(X))

make_train_data('cowslip',FLOWER_cowslip_DIR)
print(len(X))

make_train_data('crocus',FLOWER_crocus_DIR)
print(len(X))
make_train_data('Daffodil',FLOWER_Daffodil_DIR)
print(len(X))
make_train_data('Daisy',FLOWER_Daisy_DIR)
print(len(X))
make_train_data('dandelion',FLOWER_dandelion_DIR)
print(len(X))
make_train_data('fritillary',FLOWER_fritillary_DIR)
print(len(X))
make_train_data('iris',FLOWER_iris_DIR)
print(len(X))
make_train_data('lilyvalley',FLOWER_lilyvalley_DIR)
print(len(X))
make_train_data('pansy',FLOWER_pansy_DIR)
print(len(X))
make_train_data('Snowdrop',FLOWER_Snowdrop_DIR)
print(len(X))
make_train_data('sunflower',FLOWER_sunflower_DIR)
print(len(X))
make_train_data('tigerlily',FLOWER_tigerlily_DIR)
print(len(X))
make_train_data('tulip',FLOWER_tulip_DIR)
print(len(X))
make_train_data('windflower',FLOWER_windflower_DIR)
print(len(X))


#標籤對Y數組進行編碼（即Daisy-> 0，Rose-> 1等...）然後進行一次熱編碼
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,17)
X=np.array(X)
X=X/255

#分為訓練和驗證集
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

#設置隨機種子
np.random.seed(42)
rn.seed(42)
tf.random.set_random_seed(42)

#建模開始使用CNN
model = Sequential()
conv_base = VGG16(weights='imagenet', input_shape=(150,150,3), include_top=False)
conv_base.trainable=False
model.add(conv_base)


model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(17, activation = "softmax"))

batch_size=128
epochs=50

#數據擴充以防止過度擬合
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

#編譯Keras模型和摘要
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

#擬合訓練集並對驗證集進行預測
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)


# saving the model
model.save('model.pretrained.h5')

#評估模型性能
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.savefig('model_loss_pretrained.png')
plt.show()

plt.clf()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.savefig('model_acc_pretrained.png')
plt.show()
plt.clf()



['bluebell', 'buttercup', 'coltsfoot', 'cowslip', 'crocus', 'Daffodil', 'Daisy', 'dandelion', 'fritillary', 'iris', 'lilyvalley', 'pansy', 'Snowdrop', 'sunflower', 'tigerlily', 'tulip', 'windflower']


 36%|█████████████████████████████▎                                                   | 29/80 [00:00<00:00, 287.34it/s]

80


 68%|██████████████████████████████████████████████████████▋                          | 54/80 [00:00<00:00, 264.97it/s]

160


...

1200


 39%|███████████████████████████████▍                                                 | 31/80 [00:00<00:00, 301.11it/s]

1280


100%|█████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 280.15it/s]


1360
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 4, 32)          409632    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 2, 2, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 2, 2, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 64)          0         